In [1]:
#加载云盘
from google.colab import drive
drive.mount('/content/drive')
#改变工作目录
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/')
!ls

#过12个小时 空间就会被自动清理
n = len(os.listdir('trainImg/'))
if n > 10000: 
    print("train有图像{}张".format(n))
else:
    os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/trainImg')
    !ls
    !unzip -o Train.zip
    os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
backup	      lib		  testImg   Train_label.csv
clound.ipynb  submit_example.csv  trainImg
train有图像10669张


In [0]:
import tensorflow as tf
from tensorflow.train import Features, Feature, Example, Int64List, FloatList, BytesList
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import pandas as pd
import os
import time
from functools import wraps

def timeit(func):
    """@装饰器 计算func消耗的时间"""
    def wrapper(*args, **kwargs):
        start = time.time()
        ret = func(*args, **kwargs)
        cost = time.time() - start
        print(F'【{func.__name__}】{args[1]} cost {cost:.2f}S')
        return ret
    return wrapper 


"""****************************************"""
""" 处理图片                 """
"""****************************************"""
TRAIN_IMG_DIR = 'trainImg/' #训练图片的目录
TEST_IMG_DIR = 'testImg/'  #测试图片的目录
TRAIN_LIST_FILE = 'Train_label.csv' #训练图片文件名和label的列表文件
X_SHAPE = [416, 416, 3] # 预处理后图像的大小

class Pic:
    @staticmethod
    def read_image(path):
        """
        @@@ 读图片  
        :params
            path: 文件路径
        :return 
            PIL.Image类型的图片
        """
        try:
            img = Image.open(path) # img.mode = 'RGB'
        except Exception as e:
            print('[path] error...')
            raise e
        if img.mode != 'RGB':
            img = img.convert("RGB") #读取图片的过程中如果遇到非'RGB'就转换格式
        return img     

    @staticmethod
    def show_double_img(before, after, num=3):
        """
        @@@ 并列显示处理前和处理后的图片 用于比较
        :param
            before: list 处理前PIL.Image 
            after: list 处理后PIL.Image
            num:一次显示几行 
        :return
            None
        """
        plt.figure(figsize=(14, 10), dpi=75, ) #画布 宽1400 高700
        for i in range(len(before)):
            p = i % num + 1 #在第几个小图显示
            plt.subplot(2, num, p)
            plt.imshow(before[i])        
            plt.subplot(2, num, p + num)
            plt.imshow(after[i])          
            if (i+1)%num == 0: 
                plt.show()            
        
    @staticmethod
    def preprocessing(img):
        """
        @@@ 对图像数据进行预处理
        :param
            img: Image格式的图片
        :return
            norm后的array [H,W,3]，值范围[-0.5,-0.5]
        """
        #其他处理。。。

        #resize crop变成H*W大小
        img = Pic._resize(img)
        #PIL Image转换为array
        img_array = np.asarray(img, np.float32)

        # img_norm = tf.image.per_image_standardization(timg) #tensorflow中对图像标准化预处理的API
        img_norm = img_array/255. - 0.5  #[-0.5,0.5]归一化
        return img_norm

    @staticmethod
    def _resize(img, region='small'):
        """
        @@@ 对图像裁切 缩减
        :param
            img: PIL.Image
            region: 裁剪区域 ‘small'取原图最小范围
        :return
            PIL.Image 预定义大小的图片 
        """
        #四周裁剪掉的百分比(基于裁剪后的尺寸) ‘big'最接近原图
        if region == 'small':
            left, top, right, bottom = 0.1, 0.08, 0.08, 0.25 
        elif region == 'middle':
            left, top, right, bottom = 0.05, 0.03, 0.05, 0.12 
        elif region == 'big':
            left, top, right, bottom = 0.03, 0.02, 0.03, 0.06 
        else:
            left, top, right, bottom = 0, 0, 0, 0 #不裁边
        #最终尺寸
        H, W = X_SHAPE[:2]
        #resize后的图片尺寸
        exH = int(H * (1 + top + bottom)) #高
        exW = int(W * (1 + left + right)) #宽
        h, w = img.size[::-1]  #原始图片的高 宽
        new_size = None #用来缩放的新尺寸（宽高）img.resize用宽在前面
        if h > w:  #如果原图宽小
            if w > exW:  #图片超大
                new_size = (exW, int(h*exW/w)) #宽缩到resize尺寸 高相应缩
            elif w < W:  #图片超小
                new_size = (W, int(h*W/w))  #宽放大到最终尺寸 高同步放大
            else: image = img  #图片不大不小 不变
        else: #原图高比较小
            if h > exH:  #图片超大
                new_size = (int(w*exH/h), exH)
            elif h < H: #图片超小
                new_size = (int(w*H/h), H)
            else: image = img
            
        if new_size: #改变图片大小
            image = img.resize(new_size, Image.ANTIALIAS) #压缩质量 改更好的？ 
                
        #对resize好的图片 进行裁剪，满足预定义尺寸H,W （根据天空的实际图片确定的策略）
        h, w = image.size[::-1] #resize后的尺寸
        top = int(top / (1+top+bottom) * (h-H)) #上高裁切 
        left = int(left / (1+left+right) * (w-W)) #左边裁切
        image = image.crop((left, top, left+W, top+H)) #四周裁剪 留下H*W区域
        return image

    @staticmethod
    def get_train_NLs(lkind, ratio=None):
        """
        @@@ 获取train所有的image的列表(name,label)
        :param
            lkind: 'single'单label, 'multi'多label 
        :return 
            dataframe(shuffled)
        """
        df = pd.read_csv(TRAIN_LIST_FILE)
        if lkind == 'single':  #获取单label的(图片名和label列表)
            NLs = df[df.apply(lambda x: len(x['Code'])<=2, axis=1)].copy()
        elif lkind == 'multi':  #多label  
            NLs = df[df.apply(lambda x: len(x['Code'])>2, axis=1)].copy()
        else:
            raise RuntimeError('输入错误 lkind= "{}"'.format(lkind) )
        #文件名上添加路径
        NLs['FileName'] = NLs['FileName'].apply(lambda x: TRAIN_IMG_DIR + x)    
        #打乱顺序 frac是要返回的比例 1=100%
        NLs = NLs.sample(frac=1).reset_index(drop=True) 
        if ratio:  #为了应付colab经常删文件，故缩小样本节约时间,正常不要这样操作
            return NLs[:int(ratio * len(NLs))]
        else: 
            return NLs

    @staticmethod
    def get_class_num():
        """
        @@@ 获取单标签的分类总数 也就是最后层神经元数目
        :return
            int
        """
        NLs = Pic.get_train_NLs('single')
        return len(NLs['Code'].unique())
    


In [0]:
"""****************************************"""
""" 处理数据                 """
"""****************************************"""
NUM_PER_TFRECOARD = 2000 #每个tfrecoard文件包括的图片数
CLASS_NUM = Pic.get_class_num()  #单标签 标签类别数目
TFR_DIR = 'lib/'  #tfrecoard文件的存放路径
#用colab时 每次都重新生成文件 可以考虑缩小样本 0.1=1/10 None=不缩小
COLAB_PIC_RATIO = 1 
BATCH_SIZE = 16  #批量读取 1bacth多少张照片
LABEL_CLASS = {  #train数据集的分类
            'xs' : '单label训练集',
            'vs' : '单label验证集',
            'xm' : '多label训练集',           
            'vm' : '多label验证集',    
    }
STEP_PER_EPOCH = {}  #一个epoch需要多少步 总图片/batch_size

class DatasetGenerator:
    def __init__(self):
        #单label训练集和验证集的文件列表
        self.x_single_NLs, self.v_single_NLs = self.__separete(Pic.get_train_NLs('single', COLAB_PIC_RATIO))
        #多label训练集和验证集的文件列表
        self.x_multi_NLs, self.v_multi_NLs = self.__separete(Pic.get_train_NLs('multi', COLAB_PIC_RATIO))

        #检查文件夹的tfrecoard文件，若没有则生成新的
        self.check_tfrecoard()
        #计算step_per_epoch  fit时候用
        self.__calc_step_per_epoch()
        #标记（临时策略） 用于_deal_label 区分单label还是多label
        self.__now_working_class = None
        #标记 用于_parse_function 返回只包括img和label，还是包括全部信息
        self.__dataset_for_train = True #True 表示返回训练用的格式(img label)
        
    def check_tfrecoard(self):
        """
        @@@ 检查文件夹的tfrecoard文件，若没有则生成新的
        :param
            None
        :return 
            None
        """       
        #类别名  单label训练集和验证集   多label训练集和验证集 
        classes = ['xs', 'vs', 'xm', 'vm']
        #检查每一个数据集的tfrecoards是否存在
        for c in classes:
            fnames = self.__get_tfr_file_names(lclass=c) #对应的文件名
            # print('{}检查 TFRecoard:{}'.format(c, fnames))
            for f in fnames:
                if not os.path.isfile(f):
                    #只要有一个文件不存在，就全部重新生成
                    print('【{}】不存在，"{}"所有tfrecoard重新生成[{}个*{}张/个]...'.format(
                        f, LABEL_CLASS[c], len(fnames), NUM_PER_TFRECOARD))
                    try:
                        self._create_class_tfrecoards(c) #单个数据集的tfrecoard全部重新生成
                    except Exception as e:
                        raise e(f'创建{f}失败！')
                    else:
                        break #已全部重新生成 后面的文件不用检查了
                        
            print('对【"{}"{}】的{}个TFRecoard，检查完毕！'.format(c, LABEL_CLASS[c], len(fnames)))
        print('OK! 【{}】的所有TFRecoard文件检查完毕！\n'.format(classes))
        
    def _create_class_tfrecoards(self, lclass):
        """
        @@@ 把当前数据集图片列表 生成tfrecoard文件 
        :param
            lclass: train数据集的分类
        :return 
            True成功  False失败
        """     
        assert lclass in LABEL_CLASS.keys(), "lcalss 输入error..."
        #当前数据集下的 图片列表
        df = self.__get_NLs_by_class(lclass)
        #应该产生的文件名
        fnames = self.__get_tfr_file_names(lclass=lclass)
        print('"{}"：{}张图片 准备create TFRecoard:{}'.format(lclass, len(df), fnames))
        
        start = 0
        i = 0 #文件名序号
        while start < len(df):
            fname = fnames[i]  #当前写入的文件名
            df_slice = df[start:start+NUM_PER_TFRECOARD]
            if self.__create_tfrecoard(fname, df_slice):
                print('create TFRecoard[{}]  从df[{}:]... Success'.format(fname, start))
                i += 1
                start += NUM_PER_TFRECOARD
            else:
                print('create TFRecoard[{}]  从df[{}:]... Faile'.format(fname, start))
                return False
        return True        

    @timeit    
    def __create_tfrecoard(self, tfrname, df, option=None):
        """
        @@@ 把输入的图片列表 生成一个tfrecoard文件  若存在则覆盖
        :param
            tfrname：生成的tfrecoard文件 名
            df：要保存的图片 FileName和lable的列表
        :return 
            True成功  False失败
        """            
        print('---- 准备创建tfrecoard文件【{}】...'.format(tfrname, len(df)))
        ok_num = 0  #成功写入tfrecoard的图片数目
        #考虑用 with tf.io.TFRecordWriter(tfrname, options=option) as writer:
        trf_writer = tf.io.TFRecordWriter(tfrname, options=option)
        for index, (fname, label) in df.iterrows():
            try:
                img = Pic.read_image(fname)
            except:
                print(f'---- {fname}不存在，跳过...')
                continue  #文件不存在 忽略 处理下一个
            try:
                img = Pic.preprocessing(img) #裁剪 归一化等预处理
                x_shape = img.shape #处理后的图片尺寸 等于输入到模型中的图片尺寸
                assert x_shape == tuple(X_SHAPE), "准备写入TRF的图片尺寸不等于{}".format(X_SHAPE)
                img = img.reshape(-1) #变成一维
                # img_list = img.tolist()  
                label = [int(x) for x in label.split(';')]
                #对多label的情况，把label固定到5个长度（batch要统一长度）  即一张图片最多5种云
                if len(label) > 1:
                    label += [0,0,0]
                    label = label[:5]
                name = [tf.compat.as_bytes(fname)]
                #内层feature编码方式
                # print('内层feature编码', name, label)
                feature_internal = {
                            'image_raw' : Feature(float_list = FloatList(value=img)), #图形数据
                            'name'  : Feature(bytes_list = BytesList(value=name)), #路径文件名
                            'img_shape' : Feature(int64_list = Int64List(value=x_shape)),  
                            'label'   : Feature(int64_list = Int64List(value=label)),  #字符串的label   
                            }
                #使用tf.train.Example将features编码数据封装成特定的PB协议格式
                example = Example(features=Features(feature=feature_internal))
                #将序列化为字符串的example数据写入协议缓冲区
                trf_writer.write(example.SerializeToString())
            except:
                print(f'---- {fname}图片处理 保存过程中出错.')
                return False
            else:
                ok_num += 1
        print(f'---- successfully created [{tfrname}]，共处理图片{ok_num}张')
        #关闭TFRecords文件操作接口    
        trf_writer.close() 
        return True
        
    @timeit    
    def read_data_from_TFRecoard(self, lclass, batch_size=BATCH_SIZE):
        """
        @@@ 获取数据集数据 从tfrecoard里读取
        :param
            lclass: train数据集的分类
        :return 
            dataset  
            [batch,H,W,3] if batch， 3D[H,W,3] if no batch
            None if no file to read
        """
        assert lclass in LABEL_CLASS.keys(), "lcalss 输入error..."
        fnames = self.__get_tfr_file_names(lclass=lclass) #文件名列表
        self.__now_working_class = lclass  #传递给__deal_label()
        
        tfr_files = [] #待读取的tfrecoard文件列表
        for f in fnames:
            if os.path.exists(f): tfr_files.append(f)
        if tfr_files == []: return None
          
        print('从【{}...】读取{}个TFRecoard...'.format(tfr_files[0], len(tfr_files)))
        dataset_raw = tf.data.TFRecordDataset(tfr_files)
        # Set the number of datapoints you want to load and shuffle 
        dataset = dataset_raw.shuffle(buffer_size = NUM_PER_TFRECOARD//20)
        #To decode the message use the tf.train.Example.FromString method.
        #example_proto = tf.train.Example.FromString(serialized_example)
        #执行解析函数 得到数据集    
        dataset = dataset.map(self.__parse_function)
        # 不加参数=无限重复数据集
        dataset = dataset.repeat()                   
        # 定义batchsize大小
        if batch_size: #batch时 必须保证各元素长度都一样
            dataset = dataset.batch(batch_size)
        return dataset

    def iterator_from_tfrecoard(self, lclass, batch_size=BATCH_SIZE):
        """
        @ 获取iterator 数据集数据从tfrecoard里读取
        :param
            lclass: train数据集的分类
        :return 
            interator 可迭代的image label数据
            [batch,H,W,3] if batch， 3D[H,W,3] if no batch
            None if no file to read
        """
        dataset = self.read_data_from_TFRecoard(lclass, batch_size)
        iterator = dataset.make_one_shot_iterator()
        # return iterator
        if self.__dataset_for_train:  #训练时 只用（img, label）
            img, label = iterator.get_next()
            return (img, label)
        else:
            img, label, name = iterator.get_next()
            return (img, label, name)
#             
#         img = tf.reshape(img, [BATCH_SIZE]+X_SHAPE)
#         label = tf.reshape(label, [BATCH_SIZE]+[CLASS_NUM])
# 
#         if self.__dataset_for_train: 
#             while True:       
#                 yield (img, label)
#         else:
#             while True:
#                 yield (img, label, name)


    def __parse_function(self, example_proto):
        """
        @@@ 解析函数  解析dataset的每一个Example
        :param 
            example_proto: 单个example序列化后的样本
        :return
            img: array[H,W,C]
            name: b''
            label: [int]
        """
        feature_description  = {
            'image_raw' : tf.io.VarLenFeature(dtype=tf.float32),
            'name' : tf.io.VarLenFeature(dtype=tf.string),
            'img_shape' : tf.io.FixedLenFeature(shape=(3,), dtype=tf.int64),
            'label' : tf.io.VarLenFeature(dtype=tf.int64)
            }
        # 把序列化样本和解析字典送入函数里得到解析的样本
        parsed_example = tf.io.parse_single_example(example_proto, feature_description ) #返回字典
        # 解码 
        img = tf.sparse.to_dense(parsed_example['image_raw'], default_value=0) # 稀疏->密集表示

#         img = tf.reshape(img, parsed_example['img_shape']) # 转换为原来形状
        #没搞定 用tf返回 DatasetV1Adapter shapes: (?, ?, ?) 先用固定尺寸来转换
        img = tf.reshape(img, X_SHAPE) # 转换为原来形状
        label = self.__deal_label(parsed_example['label'])
        name = tf.sparse.to_dense(parsed_example['name'], default_value='')
        name = tf.squeeze(name,)
         #如果有batch 则必须保证各item返回各元素长度都一样，才能加入一个batch内   
        if self.__dataset_for_train:
            return img, label 
        else:
            return img, label, name    


    def __deal_label(self, label):
        """
        @@@ 把原始字符串的label 变成int的list
        :param
            label：原始label 单label 或者多label(e.g. 1:12)
        :return 
            list[int]  
        """
        lclass = self.__now_working_class  #现在正在处理哪个数据集
        assert lclass in LABEL_CLASS.keys(), "lcalss 输入error..."
        label = tf.sparse.to_dense(label, default_value=0)
        if lclass == 'xs' or lclass == 'vs': #如果是单label 直接转换为one-hot
            print()
            label = tf.one_hot(label[0], CLASS_NUM)
        else:  #如果是多label  还没想好
            print('现在是多label数据集"{}"，暂时不处理label'.format(lclass))
        return label       

    def __separete(self, df):
        """
        @@@ 把数据集分割成训练集和验证集 
        :return
            x,v 按比例划分的训练集和验证集
        """
        ratio = 0.7  #训练集的大小 70%
        truncation = int(len(df) * ratio) #训练集和验证集分割点
        x = df[:truncation] #训练集
        v = df[truncation:] #验证集
        return x,v
    
    def __get_NLs_by_class(self, lclass):
        """
        @@@ 通过lclass 获取对应数据集的图片 FileName和lable的列表
        :param
            lclass: 'xs' 'vs' 单label训练集和验证集 
                'xm' 'vm' 多label训练集和验证集 
        :return
            dataframe
        """
        xs = self.x_single_NLs
        vs = self.v_single_NLs
        xm = self.x_multi_NLs
        vm = self.v_multi_NLs
        try:
            df = eval(lclass)
        except:
            raise RuntimeError('输入错误 lclass= "{}"'.format(lclass))
        return df.reset_index(drop=True)

    def __get_tfr_file_names(self, lclass):
        """
        @@@ 获取各种数据集应该产生的tfrecoard文件的列表
        :param
            lclass: 'xs' 'vs' 单label训练集和验证集 
                'xm' 'vm' 多label训练集和验证集 
        :return 
            list tfrecoard文件名列表
        """
        df = self.__get_NLs_by_class(lclass) #图片名列表
        file_num = (len(df)-1) // NUM_PER_TFRECOARD + 1

        #设置的文件名 (考虑加路径 暂时同一目录下)
        names = [TFR_DIR + lclass + '_' + str(x+1) + '.tfrecord' for x in range(file_num)]
        return names
    
    def __calc_step_per_epoch(self):
        """
        @@@ 计算每个label class的一个epoch需要几步能遍历
        :param
            None
        :return 
            None 保存到
        """    
        lclass = LABEL_CLASS.keys()
        for label in lclass:
            NLs = self.__get_NLs_by_class(label)
            #简单一点 暂时用dataframe的列表，而不是真正的文件里面的图片数，实际上应该差不多
            pic_num = len(NLs)  #对应数据集的图片数
            STEP_PER_EPOCH[label] = pic_num // BATCH_SIZE + 1
        
    def __get_clound_name(self, labels):
        """
        @@@ 根据label 获取对应云的名字
        :param
            labels: int的list
        :return 
            list 云的名字
        """        
        clound = {    #编号（type）  云状类型
                0:'', #仅填充用
                1:'中云-高积云-絮状高积云',
                2:'中云-高积云-透光高积云',
                3:'中云-高积云-荚状高积云',
                4:'中云-高积云-积云性高积云',
                5:'中云-高积云-蔽光高积云',
                6:'中云-高积云-堡状高积云',
                7:'中云-高层云-透光高层云',
                8:'中云-高层云-蔽光高层云',
                9:'高云-卷云-伪卷云',
                10:'高云-卷云-密卷云',
                11:'高云-卷云-毛卷云',
                12:'高云-卷云-钩卷云',
                13:'高云-卷积云-卷积云',
                14:'高云-卷层云-匀卷层云',
                15:'高云-卷层云-毛卷层云',
                16:'低云-雨层云-雨层云',
                17:'低云-雨层云-碎雨云',
                18:'低云-积云-碎积云',
                19:'低云-积云-浓积云',
                20:'低云-积云-淡积云',
                21:'低云-积雨云-鬃积雨云',
                22:'低云-积雨云-秃积雨云',
                23:'低云-层云-碎层云',
                24:'低云-层云-层云',
                25:'低云-层积云-透光层积云',
                26:'低云-层积云-荚状层积云',
                27:'低云-层积云-积云性层积云',
                28:'低云-层积云-蔽光层积云',
                29:'低云-层积云-堡状层积云',
                }
        names = []
        for label in labels:
            names.append(clound[label])
        return names

    def _test_read_write(self):
        """@测试整个流程"""
        sess =  tf.compat.v1.InteractiveSession() 
        sess.run(tf.compat.v1.global_variables_initializer())
        
        self.__dataset_for_train = False #flag dataset读取全部类别
        classes = ['xs', 'vs', 'xm', 'vm'][3:]
        for c in classes:
            print('测试{} "{}"tfrecoard...单文件最多{}张图片  BATCH：{}'.format(
                c, LABEL_CLASS[c], NUM_PER_TFRECOARD, BATCH_SIZE))
            dataset = dg.read_data_from_TFRecoard(c)
            iter = tf.compat.v1.data.make_one_shot_iterator(dataset)
            imgs, labels, names = iter.get_next()
            # self.__test_print(imgs, labels, names)

    def _test_resize(self):
        """@测试 resize函数""" 
        before, after = [], [] 
        df = self.__get_NLs_by_class('vm')[:110]
        for index, (fname, label) in df.iterrows():
            img = Pic.read_image(fname)
            before.append(img)
            ret = Pic._resize(img, region='small')
            after.append(ret)
        Pic.show_double_img(before, after)


    def __test_print(self, imgs, labels, names): 
        """@测试 显示iterator的元素"""         
        p_freq = {} #打印频率 
        try:
            for i in range(2):
                image = imgs.eval()+0.5
                name = names.eval()
                label = labels.eval()
                print('  batch {}: '.format(i+1))
                for j in range(image.shape[0])[:]:
                    p_freq[name[j]] = p_freq.get(name[j], 0) + 1
                    print('  No.{} image name:{}  label:{} '.format(j+1, name[j], label[j]  ))
                    # xname = name[j].decode('utf-8')  #图片上显示的xlable名字
                    # for n in self.__get_clound_name(label[j]):
                    #     xname += " " + n   
                    # plt.xlabel(xname, fontproperties="SimHei")
                    plt.imshow(image[j])
                    plt.show()
        except tf.errors.OutOfRangeError:
            print("end!读完了...")        

        print('{}张图片调用频率：\n'.format(len(p_freq)))
        for key, value in p_freq.items():
            print('{value} : {key}'.format(key = key, value = value))
        
    

In [0]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, AveragePooling2D, Dropout, Concatenate, Add, ZeroPadding2D, UpSampling2D
from tensorflow.keras.layers import Flatten, BatchNormalization, Activation, Input, SeparableConv2D, LeakyReLU
from tensorflow.keras.models import Model, Sequential, load_model 
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# weights_kernel = os.path.join(TFR_DIR, 'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

EPOCHS = 20  #fit训练次数

def showinfo(func):
    """@装饰器 显示辅助信息 shape param等"""
    def wrapper(*args, **kwargs):
        # print('调用', func.__name__, args[1], 'args:',args, 'kw:',kwargs)
        res = func(*args, **kwargs)
        return res
    return wrapper

class MyModel():
    def __init__(self):
        self.model = self.create_model() 
        #checkpoint保存的文件路径
        self.cp_path = 'backup/weights-{epoch:02d}-{val_acc:.3f}.hdf5' 
        self.cp_dir = os.path.dirname(self.cp_path)

    """
    model的组件
    """
    @showinfo
    @wraps(Conv2D)
    def __Conv2D(self, x, *args, **kwargs):
        """
        @ 2D卷积层
        :param
            x: input 
            args[0]: Conv2D filters
            args[1]: Conv2D kernels
            kwargs: 可选 strides ...
        :return 
            Output tensor after applying `Conv2D` 
        """ 
        conv_kwargs = {'kernel_regularizer': regularizers.l2(5e-4)} #加点正则化 
        #'strides'==(2,2) 相当于pool
        conv_kwargs['padding'] = 'valid' if kwargs.get('strides')==(2,2) else 'same' #最好还是传'padding'
        conv_kwargs.update(kwargs)
        return Conv2D(*args, **conv_kwargs)(x)

    @showinfo
    def __Conv2D_BN(self, x, *args, **kwargs):
        """
        @ 2D卷积层 + batch-normalization + LeakyReLU
        :param
            x: input 
            args[0]: Conv2D filters
            args[1]: Conv2D kernels
        :return 
            Output tensor after applying `Conv2D` and `BatchNormalization`
        """  
        x = self.__Conv2D(x, *args, **kwargs) 
        x = BatchNormalization()(x)
        activation = kwargs.get('activation')
        if not activation: 
            x = LeakyReLU(alpha=0.1)(x) #默认使用LeakyReLU
        else:
            print('#使用:', activation )
            x = Activation(activation)(x)
        return x

    @showinfo
    def __res_blocks(self, x, filters, num_blocks, name):
        """
        @ A series of resblocks starting with a downsampling Convolution2D
        :param
            x: input 
            filters: 卷积核数量
            num_blocks: 残差block数量
        :return 
            Output tensor  
        """ 
        # uses left and top padding instead of 'same' mode
        x = ZeroPadding2D(((1,0),(1,0)))(x)
        #图片尺寸 缩小一半
        x = self.__Conv2D_BN(x, filters, (3,3), strides=(2,2))
        #多个残差块
        for i in range(num_blocks):
            with tf.name_scope(name+str(i)):
                y = self.__Conv2D_BN(x, filters//2, (1,1))
                y = self.__Conv2D_BN(y, filters, (3,3))
                x = Add()([x,y])  #residual shoutcut
        return x

    """
    darknet骨干网络
    """
    def darknet_body(self, input):
        """
        @@@ Darknent body having 52 Convolution2D layers
        :param
            input: input tensor
        :return 
            Output tensor  
        """         
        x = self.__Conv2D_BN(input, 32, (3,3))
        x = self.__res_blocks(x, 64, 1, 'Residual_Block_1_')
        x = self.__res_blocks(x, 128, 2, 'Residual_Block_2_')
        x = self.__res_blocks(x, 256, 8, 'Residual_Block_3_')
        x = self.__res_blocks(x, 512, 8, 'Residual_Block_4_')
        x = self.__res_blocks(x, 1024, 4, 'Residual_Block_5_')
        return x    

    def __make_last_layer(self, input, filters, out_filters):
        """
        @ 6 Conv2D_BN_Leaky layers followed by a Conv2D_linear layer
        :param
            input: input tensor
            filters: 卷积核数目
            out_filters: y输出卷积核数目（通道数）
        :return 
            x, y: 两个Output tensor  
        """ 
        x = self.__Conv2D_BN(input, filters, (1,1))
        x = self.__Conv2D_BN(x, filters*2, (3,3))
        x = self.__Conv2D_BN(x, filters, (1,1))
        x = self.__Conv2D_BN(x, filters*2, (3,3))
        x = self.__Conv2D_BN(x, filters, (1,1))

        y = self.__Conv2D_BN(x, filters*2, (3,3))
        y = self.__Conv2D(y, out_filters, (1,1))
        return x, y

    def fpn(self, net, *args, **kwargs):
        """
        @ feature pyramid networks 特征金字塔网络FPN
        #同时利用低层特征高分辨率和高层特征的高语义信息，通过融合这些不同层的特征达到预测的效果。
        #并且预测是在每个融合后的特征层上单独进行的
        :param
            net: 骨干网络model
        :return 
            [Output-tensor,] 效果：所有尺度下的特征都有丰富的语义信息 
        """  
        #以每个‘stage’为一个pyramid level，取每个stage最后layer输出的feature map作为pyramid level
        #对应darknet,是stage2 stage3 stage4 stage5的res block各自最后一个输出
        #stage2 stage3 stage4 stage5的res block各自最后一个输出
        darknet = net  #input[416,416]
        C5 = darknet.layers[184].output  #stage5输出 [13,13,1024]
        C4 = darknet.layers[152].output  #stage4输出 [26,26,512]
        C3 = darknet.layers[92].output  #stage3输出 [52,52,256]
        C2 = darknet.layers[32].output  #stage2输出 [104,104,128]

        # P2，P3，P4，P5
        out_filters = 256  #输出通道数 都一样
        P5 = self.__Conv2D(C5, out_filters, (1,1), name='fpn_p5')
        P4 = Add()([  #横向连接
            UpSampling2D(2)(P5),  #上采样跟C4同尺寸
            self.__Conv2D(C4, out_filters, (1,1)),  #1*1卷积跟P5通道数一样
            ])
        P3 = Add()([   
            UpSampling2D(2)(P4),
            self.__Conv2D(C3, out_filters, (1,1)),
            ])
        P2 = Add()([   
            UpSampling2D(2)(P3),
            self.__Conv2D(C2, out_filters, (1,1)),
            ])
        # P2-P4最后又做了一次3*3的卷积，作用是消除上采样带来的混叠效应
        P4 = self.__Conv2D(P4, out_filters, (3,3), name='fpn_p4')
        P3 = self.__Conv2D(P3, out_filters, (3,3), name='fpn_p3')
        P2 = self.__Conv2D(P2, out_filters, (3,3), name='fpn_p2')
        # 最后得到了融合了不同层级特征的特征图列表
        rpn_feature_maps = [P2, P3, P4, P5]
        return rpn_feature_maps


    def create_model(self):
        """
        @ 创建 模型
        :param
            None 
        :return 
            model
        """
        model_input = Input(shape=X_SHAPE) 
        #darknet 53成网络模型
        darknet = Model(model_input, self.darknet_body(model_input)) 

        #！！！加载darknet 预训练参数

        P2, P3, P4, P5 = self.fpn(darknet)
        print(P2, P3, P4, P5)

        x = AveragePooling2D()(darknet.output)
        x = Flatten()(x)

        pridictions = Dense(CLASS_NUM, activation='softmax')(x)
        # 构建我们需要训练的完整模型
        model = Model(model_input, pridictions)

        # 编译模型（一定要在锁层以后操作）
        model.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['acc'],
               )            
        return model
    
    def fit(self, xs, vs):
        """
        @@@ model fit
        :param
            xs: 训练集
            vs: 验证集
        :return 
            history 对象。其 History.history 属性是连续 epoch 训练损失和评估值，以及验证集损失和评估值的记录（如果适用）
        """    
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=2)

        checkpoint = ModelCheckpoint(filepath=self.cp_path,
                        verbose=1,
                        save_weights_only = True,
                        mode = 'max',
                        load_weights_on_restart = True,
                        )
        hist = self.model.fit(xs,
                    epochs=EPOCHS,
                    verbose=1,
                    steps_per_epoch=STEP_PER_EPOCH['xs'],
                    validation_data=vs,
                    validation_steps=STEP_PER_EPOCH['vs'],
                    callbacks=[early_stopping, checkpoint],  
                    )
        self.model.save_weights('my_model_weights.h5') #最后保存一次
        return hist



In [0]:
# model = MyModel()
# model.model.summary()
# for i, layer in enumerate( model.model.layers):
#     print(i, layer)
# plot_model(model.model, to_file='model.png')

In [0]:
dg = DatasetGenerator()
#     print(STEP_PER_EPOCH)

XS_dataset = dg.read_data_from_TFRecoard('xs') #单label训练集
VS_dataset = dg.read_data_from_TFRecoard('vs')

m = MyModel()
m.fit(XS_dataset, VS_dataset)

# img, label = dg.iterator_from_tfrecoard('xs') #测试用


# dg._test_read_write()
# dg._test_resize()
#     t = dg._separete(Pic.get_train_NLs('single'))


#     #比较resize前后的图片
#     df = dg._test_get_img('xs', 15)
#     before, after = [], []
#     for index, (name, label) in df.iterrows():
#         img = Pic.read_image(name)
#         before.append(img)
#         img = Pic._resize(img)
#         after.append(img)
#     Pic.show_double_img(before, after, 3)

【lib/xs_1.tfrecord】不存在，"单label训练集"所有tfrecoard重新生成[4个*2000张/个]...
"xs"：7147张图片 准备create TFRecoard:['lib/xs_1.tfrecord', 'lib/xs_2.tfrecord', 'lib/xs_3.tfrecord', 'lib/xs_4.tfrecord']
---- 准备创建tfrecoard文件【lib/xs_1.tfrecord】...


In [0]:
# t = m.model.evaluate(VS_dataset, verbose=1, steps=STEP_PER_EPOCH['vs'],)

In [0]:
t

In [0]:
# m.model.save_weights('my_model_weights.h5')
# m.model.load_weights('my_model_weights.h5')

In [0]:
*a = (1, 'dd', (2,3))
b,c,d = a
type(a)